<a href="https://colab.research.google.com/github/Yash-Jaiswal28/Virtual_AI_Dr/blob/main/Demo_Ai_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-groq
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4

In [2]:
import os
import pandas as pd
import numpy as np
from getpass import getpass
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate

In [18]:
GROQ_API_KEY = getpass()
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

··········


In [19]:
llm = ChatGroq(model_name="llama3-8b-8192")

In [20]:
User_Data=pd.DataFrame(columns=["Name","Phone Number","Problem","Resolution"])

In [21]:
def get_conversational_chain(complain, chat_history):
    prompt_template = """You are a medical experiment. You have been given the patient's complain. You have to
        diagnose the patient's disease. Identify and ask crucial question to help you narrow down and diagnose the
        patient. Ask only one question based on the response. If no chat-history is provided ask initial first question.

        Previous Conversation: {previous_conversation}

        Patient Complain:
        {complain}

        Question:
        """
    previous_conversation = " ".join([entry["user"] for entry in chat_history])
    prompt = prompt_template.format(previous_conversation=previous_conversation, complain=complain)
    response = llm.predict(prompt)
    return response

In [22]:
def get_final_solution(complain, chat_history):
    prompt_template ="""
        You are an expert chat doctor. Your task is to suggest the best diagnosing to the patient based on the complain and follow up questions.
        Do not suggest the steps to refer to doctor. Mention the list of possible disease, diagnose and common medications for the same. You make take help from the following contexts to diagnose the disease.
        Only Return the Final Response of expected disease. Do not return the intermediate results.


        Question: the input question you must answer
        Thought: you should always think about what to do
        Action: the action to take
        Action Input: the input to the action
        Observation: the result of the action
       ... (this Thought/Action/Action Input/Observation can repeat N times)
        Final Answer: the final answer to the original input question

        Begin!

        Patient Symptoms: {complain}

        Follow Up questions and answer from patient:
        {chat_history_str}
        """

    chat_history_str = ""
    for i, entry in enumerate(chat_history):
        if 'user' in entry:
            chat_history_str += f"User: {entry['user']}\n"
        elif 'User' in entry:
            chat_history_str += f"User: {entry['User']}\n"
        chat_history_str += f"Dr: {entry['Dr']}\n"
    prompt = prompt_template.format(chat_history_str=chat_history_str, complain=complain)
    response = llm.predict(prompt)
    return response

In [23]:
name = input("How are you I'm an AI Doctor, before we get started, please tell me your name ")
num = input("And your phone number ")
print(f"Welcome {name}, How can I help you ? ")
# complain = input("What's your complaint? ")
chat_history = []
for i in range(4):
    user = input()
    response = get_conversational_chain(user, chat_history)
    chat_history.append({"user": user, "Dr": response})
    print(response)

lastly = input()
chat_history.append({"user":lastly,"Dr":"ok, I'm done questioning"})

How are you I'm an AI Doctor, before we get started, please tell me your name Yash
And your phone number 234567
Welcome Yash, How can I help you ? 
I have red skin 
Can you tell me more about the redness of your skin? Is it a patchy redness or is the entire skin surface affected?
it is round in shape around my armpit
Interesting! Given that you have red skin and it's round in shape around your armpit, I'd like to know:

Are the red skin lesions (the round shape) tender to the touch, or are they relatively flat and non-tender?
flat
Interesting! With the patient complaining of flatness and having red, round skin lesions around the armpit, my next question would be:

Are the lesions itchy or painful?
yeah itchy
Based on the patient's symptom of having red, itchy, round, flat skin around the armpit, my next question is:

Do you have any history of allergies or previous reactions to any substances, such as insect bites, medications, or foods, that may have triggered this rash?
No


In [24]:
final_response = get_final_solution(chat_history[0]['user'], chat_history)
print(final_response)

**Final Answer:**

Based on the patient's symptoms of red skin, round in shape around the armpit, flat, itchy, and no history of allergies, I diagnose the patient with **Eczema** (Atopic Dermatitis).

**Possible Diseases:**

* Eczema (Atopic Dermatitis)
* Contact Dermatitis
* Insect Bite Allergy
* Seborrheic Dermatitis
* Psoriasis

**Common Medications:**

* Topical corticosteroids (hydrocortisone cream)
* Topical immunomodulators (pimecrolimus cream)
* Moisturizers (cetaphil cream)
* Antihistamines (diphenhydramine)
* Oral corticosteroids (prednisone) for severe cases


In [25]:
User_Data.loc[len(User_Data)] = [name, num, chat_history[0]['user'], final_response]

In [26]:
User_Data.head()

,Name,Phone Number,Problem,Resolution
0,Yash,234567,I have red skin,**Final Answer:**\n\nBased on the patient's sy...


In [27]:
User_Data.to_csv('output.csv', index=False)

In [28]:
def research(med):
    prompt_template ="""
        You are an medical expert , researcher and guide. Your task is to tell me completely about {med}
        """

    prompt = prompt_template.format(med=med)
    response = llm.predict(prompt)
    return response

In [29]:
final_response = research("metoprolol")
print(final_response)

As a medical expert, researcher, and guide, I'd be happy to provide you with a comprehensive overview of metoprolol.

**What is Metoprolol?**

Metoprolol is a type of medication known as a beta blocker, which is used to treat high blood pressure (hypertension), angina (chest pain), and certain heart conditions. It belongs to a class of medications called selective beta-1 blockers, which means it primarily targets the beta-1 receptors in the heart.

**How does Metoprolol work?**

Metoprolol works by slowing down the heart rate and reducing the force of the heart's contractions. This helps to:

1. Lower blood pressure: By reducing the heart rate and blood pressure, metoprolol helps to reduce the workload on the heart and blood vessels, which can help to lower blood pressure.
2. Reduce angina: By reducing the heart rate and blood pressure, metoprolol can help to reduce the frequency and severity of angina attacks.
3. Prevent heart failure: By reducing the heart rate and blood pressure, me